**Diabetes_012**: 0 = no diabetes 1 = prediabetes 

**HighBP**:0 = no high BP 1 = high BP

**HighChol**: 0 = no high cholesterol 1 = high cholesterol

**CholCheck**: 0 = no cholesterol check in 5 years 1 = yes cholesterol check in 5 years

**BMI**:Body Mass Index

**Smoker**: Have you smoked at least 100 cigarettes in your entire life? [Note: 5 packs = 100 cigarettes] 0 = no 1 = yes

**Stroke**: (Ever told) you had a stroke. 0 = no 1 = yes

**HeartDiseaseorAttack**: coronary heart disease (CHD) or myocardial infarction (MI) 0 = no 1 = yes

**PhysActivity**: physical activity in past 30 days - not including job 0 = no 1 = yes

**Fruits**:Consume Fruit 1 or more times per day 0 = no 1 = yes

**Veggies**: Consume Vegetables 1 or more times per day 0 = no 1 = yes

**HvyAlcoholConsump**: Heavy drinkers (adult men having more than 14 drinks per week and adult women having more than 7 drinks per week) 0 = no, 1 = yes

**AnyHealthcare**: Have any kind of health care coverage, including health insurance, prepaid plans such as HMO, etc. 0 = no 1 = yes

**NoDocbcCost**: Was there a time in the past 12 months when you needed to see a doctor but could not because of cost? 0 = no 1 = yes

**GenHlth**: Would you say that in general your health is: scale 1-5 1 = excellent 2 = very good 3 = good 4 = fair 5 = poor

**MentHlth**: 0-30, Now thinking about your mental health, which includes stress, depression, and problems with emotions, for how may days during the past 30 days did you feel these emotions

**PhysHlth**:0-30, Now thinking about your physical health, which includes physical illness and injury, for how many days during the past 30 days were you ill or injured? 

**DiffWalk**: Do you have serious difficulty walking or climbing stairs? 0 = no 1 = yes

**Sex**: 0 = female 1 = male

**Age**:13-level age category, 1 = 18-24, 2 = 25-29, 3 = 30-34, 4 = 35-39, 5 = 40-44, 6 = 45-49, 7 = 50-54, 8 = 55-59, 6 = 60-64, 7 = 65-69, 8 = 70-74, 9 = 75-79, 10 = 80-84, 11 = 85-89, 12 = 90-95 13 = 95 or older

**Education**: Education level, scale 1-6 1 = Never attended school or only kindergarten 2 = primary school, 3 = middle school, 4 = high school, 5 = Bachelor, 6= Master and + 

**Income**: Income scale  scale 1-8 1 = less than $20,000, 2 = $20,000-$29,999, 3= $30,000-$39,999, 4= $40,000-$49,999, 5= $50,000-$59,999, 6 = $60,000-$69,999, 7 = $70,000-$79,999, 8= $80,000-$89,999

**Library**

Import the necessary library 

In [3]:
%%time 
import boto3 
import os 
import sagemaker
from sagemaker.xgboost.estimator import XGBoost 
from sagemaker.session import Session 
from sagemaker.inputs import TrainingInput 
import pandas as pd 
import sklearn as sk 
import numpy as np 
import sagemaker as sg 
import matplotlib as mt 
from sklearn.model_selection import train_test_split
import re 
import urllib.request
from sklearn. datasets import load_svmlight_file

ModuleNotFoundError: No module named 'sagemaker'

**Importing the dataset**

Loading the dataset named "diabetes_binary.csv" from S3 bucket called "diabetes-prediction-b12"

In [3]:
from sagemaker import get_execution_role

role = get_execution_role()
bucket='diabetes-prediction-b12'
data_key = 'diabetes_binary.csv'
data_location = 's3://diabetes-prediction-b12/input_data/diabetes_binary.csv'.format(bucket, data_key)

df = pd.read_csv(data_location)


drop the first column that is a repeated index column 

In [4]:
df = df.drop(df.columns[0], axis=1)

Check the dataset

In [5]:
df.head()


,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
1,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
2,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
3,0.0,1.0,1.0,1.0,25.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,2.0,0.0,1.0,10.0,6.0,8.0
4,0.0,1.0,0.0,1.0,30.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,3.0,0.0,14.0,0.0,0.0,9.0,6.0,7.0


**Fine tuning**

We initially got a recall of 50%, but we would like to increase this result. One possible way was to balance the dataset as the original target variable was highly imbalance 

In [6]:

# Get the indices of the rows where Diabetes equals 0
index_names = df[df['Diabetes_012'] == 0].sample(n=100000, random_state=42).index

# Drop those rows from the dataset
df.drop(index_names, inplace=True)

In [7]:
df

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
1,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
2,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
5,0.0,1.0,1.0,1.0,25.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,0.0,0.0,1.0,0.0,11.0,4.0,4.0
6,0.0,0.0,0.0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,1.0,8.0,4.0,3.0
7,1.0,0.0,0.0,1.0,25.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,13.0,6.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206677,0.0,1.0,1.0,1.0,23.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,4.0,0.0,5.0,0.0,1.0,8.0,3.0,2.0
206678,0.0,1.0,0.0,1.0,30.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,2.0,1.0
206679,0.0,0.0,0.0,1.0,27.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,6.0,5.0
206680,1.0,1.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,1.0,0.0,11.0,2.0,4.0


Transform the necessary columns to categorical 

In [8]:
df['Diabetes_012']=df['Diabetes_012'].astype('category').cat.codes
df['HighBP']=df['HighBP'].astype('category').cat.codes
df['HighChol']=df['HighChol'].astype('category').cat.codes
df['CholCheck']=df['CholCheck'].astype('category').cat.codes
df['Smoker']=df['Smoker'].astype('category').cat.codes
df['Stroke']=df['Stroke'].astype('category').cat.codes
df['HeartDiseaseorAttack']=df['HeartDiseaseorAttack'].astype('category').cat.codes
df['PhysActivity']=df['PhysActivity'].astype('category').cat.codes
df['Fruits']=df['Fruits'].astype('category').cat.codes
df['Veggies']=df['Veggies'].astype('category').cat.codes
df['HvyAlcoholConsump']=df['HvyAlcoholConsump'].astype('category').cat.codes
df['AnyHealthcare']=df['AnyHealthcare'].astype('category').cat.codes
df['NoDocbcCost']=df['NoDocbcCost'].astype('category').cat.codes
df['DiffWalk']=df['DiffWalk'].astype('category').cat.codes
df['Sex']=df['Sex'].astype('category').cat.codes


Split the dataset into training and test set 

In [9]:
train,test= np.split(df.sample (frac=1), [int (0.8 * len (df))])

Uploading the training and test sets to S3 bucket 

In [10]:
train.to_csv('train.csv',header=False, index=False) 
test.to_csv('test.csv',header=False, index= False)

In [11]:
bucket = "diabetes-prediction-b12"

boto3.Session().resource ('s3').Bucket("diabetes-prediction-b12").Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource ('s3').Bucket("diabetes-prediction-b12").Object(os.path.join(prefix, 'test/test.csv' )).upload_file('test.csv')


Import the the training and test sets from S3 for the model 

In [12]:
from sagemaker.inputs import TrainingInput
bucket = "diabetes-prediction-b12"

s3_train_data=TrainingInput(s3_data='s3://{}/{}/train'.format(bucket,prefix),content_type= "csv")
s3_test_data=TrainingInput(s3_data='s3://{}/{}/test'.format(bucket,prefix),content_type= "csv")

Import the built-in XGboost model from Sagemaker

In [13]:
xgboost_container = sagemaker.image_uris.retrieve ("xgboost", boto3.Session().region_name, "latest")
print(xgboost_container)


813361260812.dkr.ecr.eu-central-1.amazonaws.com/xgboost:latest


The output bucket where the final model will be exported to

In [14]:
output_location='s3://{}/{}/output'.format(bucket,prefix)


**Fine tuning of hyper parameters of the XGBoost Model**


In [15]:
sess=sagemaker.Session()
role=sagemaker.get_execution_role()
# construct a SageMaker estimator that calls the xgboost-container
xgboost= sagemaker.estimator.Estimator(xgboost_container,
                                       role,
                                       instance_count=1,
                                       instance_type = "ml.m4.xlarge",
                                       output_path = output_location, sagemaker_session = sess)
xgboost.set_hyperparameters (gamme='4',
                             min_child_weight = "6",
                             subsample = "0.7",
                             objective = "binary:logistic",
                             num_round = "50")

**Training an XGBoost model using the training data stored in an S3 bucket, and evaluating the performance of the trained model on a validation set**

In [16]:
xgboost.fit({'train':s3_train_data, 'validation': s3_test_data})

INFO:sagemaker:Creating training-job with name: xgboost-2023-03-10-12-26-26-617


2023-03-10 12:26:26 Starting - Starting the training job...
2023-03-10 12:26:42 Starting - Preparing the instances for training......
2023-03-10 12:27:42 Downloading - Downloading input data...
2023-03-10 12:28:06 Training - Downloading the training image...
2023-03-10 12:28:57 Training - Training image download completed. Training in progress...Arguments: train
[2023-03-10:12:29:08:INFO] Running standalone xgboost training.
[2023-03-10:12:29:08:INFO] File size need to be processed in the node: 6.06mb. Available memory size in the node: 8605.66mb
[2023-03-10:12:29:08:INFO] Determined delimiter of CSV input is ','
[12:29:08] S3DistributionType set as FullyReplicated
[12:29:08] 85347x21 matrix with 1792287 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-03-10:12:29:08:INFO] Determined delimiter of CSV input is ','
[12:29:08] S3DistributionType set as FullyReplicated
[12:29:08] 21337x21 matrix with 448077 entries loaded from /opt/ml/input/data/vali

**Deploying the Machine Learning Model As Endpoints**

In [18]:
#Deploy Machine Learning Model As Endpoints
xgb_predictor = xgboost.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', endpoint_name='diabetes-12')


INFO:sagemaker:Creating model with name: xgboost-2023-03-10-12-31-11-370
INFO:sagemaker:Creating endpoint-config with name diabetes-12
INFO:sagemaker:Creating endpoint with name diabetes-12


------!

**Prediction of the model**

In [19]:
from sagemaker.predictor import csv_serializer
test_data_array = test.drop(["Diabetes_012"], axis=1).values #load the data into an array
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


(21337,)


In [20]:
xgb_predictor.__dict__.keys()

dict_keys(['endpoint_name', 'sagemaker_session', 'serializer', 'deserializer', '_endpoint_config_name', '_model_names', '_context'])

In [21]:
predictions_array

array([0.05133441, 0.55750173, 0.00496434, ..., 0.01770472, 0.23628007,
       0.01394187])

**Creating the confusion Matrix**

In [2]:
cm = pd.crosstab(index=test["Diabetes_012"], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Diabetes", "Diabetes"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Diabetes", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Diabetes", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
     

NameError: name 'pd' is not defined

**Invoking the end point**

In [23]:
sm_runtime = boto3.client("runtime.sagemaker")
endpoint_name = "diabetes-12"


INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


**Prediction of Diabetes Probability with different instances**

In [26]:
payload = "0, 0, 0, 25.6, 1, 0, 0, 1, 1, 1, 0, 1, 0, 3, 2, 10, 1, 1, 9, 4, 4"

response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="text/csv",
    Body=payload
)

response_str_1 = response["Body"].read().decode()

In [27]:
print(response_str_1)

0.1531713604927063


In [29]:
payload = "0, 0, 1, 21.5, 0, 0, 0, 1, 1, 1, 1, 1, 0, 4, 6, 7, 1, 0, 9, 4, 4"

response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="text/csv",
    Body=payload
)

response_str_2 = response["Body"].read().decode()

In [31]:
print(response_str_2)

0.10756269097328186


In [32]:
payload = "0,0,0,31,1,0,0,1,1,1,0,1,0,3,8,15,1,0,25,6,5"

response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="text/csv",
    Body=payload
)

response_str_3 = response["Body"].read().decode()


In [34]:
print(response_str_3)

0.041493646800518036
